## Pre-Training GPT-Neo on TinyStories
HuggingFace links: [GPT-Neo model](https://huggingface.co/docs/transformers/model_doc/gpt_neo), [TinyStories dataset](https://huggingface.co/datasets/roneneldan/TinyStories). 

In [186]:
# %pip install wandb
# %pip install transformers
# %pip install datasets
# %pip install torch torchvision torchaudio
# %pip install accelerate -U

In [187]:
with open('keys.txt', 'r') as file:
    api_key = file.read().strip()
import wandb; wandb.login(key=api_key)
import torch.nn as nn
import torch
from transformers import GPT2TokenizerFast, GPTNeoForCausalLM, GPTNeoConfig
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
from datasets import load_dataset, load_from_disk, DatasetDict
debug_mode = True
config = GPTNeoConfig(
    vocab_size=10_000,
    hidden_size=512,
    max_position_embeddings=512,
    num_layers=2,
    attention_types=[[["global", "local"], 1]],
    num_heads=4,
    window_size=256,
    intermediate_size=1024
)
# config

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


### Pre-Processing
All experiments use the same tokenizer, so in theory, we only need to preprocess our data once. This means you can subsequently skip this CPU-intensive task on the DelftBlue GPU nodes. 
(except if you are experimenting with different context lengths, in that case you will need to re-tokenise to recover potentially truncated inputs; see below).
You may even have to load the entire dataset (or pre-load chunks) into memory, DelftBlue has extremely slow IO. 

#### Tokenisation
Two things happen during tokenisation, of which the latter is optional:

1. Map (sub-)words to integers $ \in [0, 10000]$.
2. Truncate/pad each input as necessary to fit within `hidden_size`. 

In [188]:
tokenizer = GPT2TokenizerFast.from_pretrained('10k-gpt-neo', model_max_length=config.hidden_size)

# in theory, window_size x num_layers is possible. 
# But, this project is not about the efficacy of sliding window attention (though it could be!)
assert tokenizer.model_max_length == 512 
assert tokenizer.vocab_size == config.vocab_size

# printing this because of a bug in tokenizers (should be fixed now) https://github.com/huggingface/transformers/issues/26500
print(f'padding token is {tokenizer.pad_token}')
# HF wasn't saving this nor the tokenizer's pad_token
config.pad_token_id = tokenizer.pad_token_id

padding token is <|endoftext|>


In [189]:
dataset = load_dataset('roneneldan/tinystories')

small_dataset = DatasetDict({
    'train': dataset['train'].select(range(1000)),
    'validation': dataset['validation'].select(range(100))
})

if debug_mode:
    dataset = small_dataset

tokenized_dataset = dataset.map(
    lambda x: tokenizer(x['text'], truncation=True, padding='max_length'), 
    batched=True, num_proc=8, batch_size=1_000)

tokenized_dataset.save_to_disk(f'./tokenized_dataset', num_proc=5)

tokenized_dataset = load_from_disk(f'./tokenized_dataset')

assert len(tokenized_dataset['train'][0]['input_ids']) == config.hidden_size
tokenized_dataset['train'][0]['input_ids'][-10:] 
# should be eos tokens (9999), given that most short stories are <512 tokens

Repo card metadata block was not found. Setting CardData to empty.






Saving the dataset (5/5 shards): 100%|██████████| 1000/1000 [00:00<00:00, 8537.08 examples/s] 






Saving the dataset (5/5 shards): 100%|██████████| 100/100 [00:00<00:00, 1044.81 examples/s]


[9999, 9999, 9999, 9999, 9999, 9999, 9999, 9999, 9999, 9999]

### Training
#### Model
We use GPT-Neo as the architecture for our model. This consists of the following blocks, generally seen in transformer architectures:

- Embeddings: Map one-hot (sparse) token vectors to a dense vector of length `hidden_size`. Add positional encoding. 
- $n$ Blocks: Contain self-attention and FFNs.
- Head: Map hidden state back to a useful output. 

We use the config specified at the top of this notebook to construct a model. 

In [190]:
import torch.nn.functional as F

class GPTNeoSelfAttention(nn.Module):
    def __init__(self, config, attention_type):
        super().__init__()
        self.config = config

        max_positions = config.max_position_embeddings
        bias = torch.tril(torch.ones((max_positions, max_positions), dtype=bool)).view(
            1, 1, max_positions, max_positions
        )

        # local causal self attention is a sliding window where each token can only attend to the previous
        # window_size tokens. This is implemented by updating the causal mask such that for each token
        # all other tokens are masked except the previous window_size tokens.
        if attention_type == "local":
            bias = torch.bitwise_xor(bias, torch.tril(bias, -config.window_size))

        self.register_buffer("bias", bias, persistent=False)
        self.register_buffer("masked_bias", torch.tensor(-1e9), persistent=False)

        self.attn_dropout = nn.Dropout(float(config.attention_dropout))
        self.resid_dropout = nn.Dropout(float(config.resid_dropout))
        self.is_causal = True

        self.embed_dim = config.hidden_size
        self.num_heads = config.num_heads
        self.head_dim = self.embed_dim // self.num_heads
        if self.head_dim * self.num_heads != self.embed_dim:
            raise ValueError(
                f"embed_dim must be divisible by num_heads (got `embed_dim`: {self.embed_dim} and `num_heads`:"
                f" {self.num_heads})."
            )
        
        self.k_proj = nn.Linear(self.embed_dim, self.embed_dim, bias=False)
        self.v_proj = nn.Linear(self.embed_dim, self.embed_dim, bias=False)
        self.q_proj = nn.Linear(self.embed_dim, self.embed_dim, bias=False)
        self.out_proj = nn.Linear(self.embed_dim, self.embed_dim, bias=True)
        self.beta = nn.Parameter(torch.zeros(1,))
        self.ELU = nn.ELU()


    def _split_heads(self, tensor, num_heads, attn_head_size):
        """
        Splits hidden_size dim into attn_head_size and num_heads
        """
        new_shape = tensor.size()[:-1] + (num_heads, attn_head_size)
        tensor = tensor.view(new_shape)
        return tensor.permute(0, 2, 1, 3)  # (batch, head, seq_length, head_features)

    def _merge_heads(self, tensor, num_heads, attn_head_size):
        """
        Merges attn_head_size dim and num_attn_heads dim into hidden_size
        """
        tensor = tensor.permute(0, 2, 1, 3).contiguous()
        new_shape = tensor.size()[:-2] + (num_heads * attn_head_size,)
        return tensor.view(new_shape)

    def _attn(self, query, key, value, compressive_memory, z, attention_mask=None, head_mask=None):
        # Keep the attention weights computation in fp32 to avoid overflow issues
        query = query.to(torch.float32)
        key = key.to(torch.float32)

        sigma_q = self.ELU(query[:, :, :, :]) + 1.0
        sigma_k = self.ELU(key[:, :, :, :]) + 1.0

        A_mem = ((sigma_q @ compressive_memory) / ((sigma_q @ z) + 1e-6))
        A_dot = query[:, :, :, :] @ key[:, :, :, :].transpose(-2, -1)

        query_length, key_length = query.size(-2), key.size(-2)
        causal_mask = self.bias[:, :, key_length - query_length : key_length, :key_length]
        mask_value = torch.finfo(A_dot.dtype).min
        # Need to be a tensor, otherwise we get error: `RuntimeError: expected scalar type float but found double`.
        # Need to be on the same device, otherwise `RuntimeError: ..., x and y to be on the same device`
        mask_value = torch.tensor(mask_value, dtype=A_dot.dtype).to(A_dot.device)
        A_dot = torch.where(causal_mask, A_dot, mask_value)

        if attention_mask is not None:
            # Apply the attention mask
            A_dot = A_dot + attention_mask

        # attn_weights = nn.functional.softmax(attn_weights, dim=-1)
        A_dot = A_dot.to(value.dtype)
        A_dot = self.attn_dropout(A_dot)

        # Mask heads if we want to
        if head_mask is not None:
            A_dot = A_dot * head_mask
            
        A_dot = nn.functional.softmax(A_dot / torch.sqrt(torch.tensor(self.head_dim)), dim = -1)




        A_dot = torch.matmul(A_dot, value)
        attention_output = (F.sigmoid(self.beta) * A_mem) + ((1 - F.sigmoid(self.beta)) * A_dot)

        delta = (sigma_k @ compressive_memory) / ((sigma_k @ z) + 1e-6)
        compressive_memory = compressive_memory + (sigma_k.transpose(-2, -1) @ (value[:, :, :, :] - delta))

        z = z + sigma_k.sum(dim = -2, keepdim = True)

        return attention_output, A_dot

    def forward(
        self,
        hidden_states,
        attention_mask=None,
        layer_past=None,
        head_mask=None,
        use_cache=False,
        output_attentions=False,
    ):
        query = self.q_proj(hidden_states)
        key = self.k_proj(hidden_states)
        value = self.v_proj(hidden_states)
        device = query.device
        query = self._split_heads(query, self.num_heads, self.head_dim)
        key = self._split_heads(key, self.num_heads, self.head_dim)
        value = self._split_heads(value, self.num_heads, self.head_dim)

        compressive_memory = torch.zeros((self.num_heads, self.head_dim, self.head_dim), device=device)
        z = torch.zeros((self.num_heads, self.head_dim, 1), device=device)

        if layer_past is not None:
            past_key = layer_past[0]
            past_value = layer_past[1]
            key = torch.cat((past_key, key), dim=-2)
            value = torch.cat((past_value, value), dim=-2)

        if use_cache is True:
            present = (key, value)
        else:
            present = None

        attn_output, attn_weights = self._attn(query, key, value, compressive_memory, z, attention_mask, head_mask)

        attn_output = self._merge_heads(attn_output, self.num_heads, self.head_dim)
        attn_output = self.out_proj(attn_output)
        attn_output = self.resid_dropout(attn_output)

        outputs = (attn_output, present)
        if output_attentions:
            outputs += (attn_weights,)

        return outputs  # a, present, (attentions)

In [191]:
class CustomGPTNeoForCausalLM(GPTNeoForCausalLM):
    def __init__(self, config):
        super().__init__(config)
        # Assuming attention types are specified in config.attention_layers
        # Replace the standard attention with the specified type
        for block in self.transformer.h:
            block.attn.attention = GPTNeoSelfAttention(config, block.attn.attention_type)
model = CustomGPTNeoForCausalLM( 
    config = config,
)
print(f'The model has {model.num_parameters():,} parameters.')


The model has 9,585,666 parameters.


In [192]:
# # model = CustomGPTNeoForCausalLM( 
# #     config = config,
# # )
# model = GPTNeoForCausalLM(config=config)

# print(f'The model has {model.num_parameters():,} parameters.')
# # Token embeddings:    10'000 * 512 = 5M 
# # Position embeddings:    512 * 512 = 260K
# # 2 Blocks: 2 * (~1M + ~1M) = 4M
# #   Attention: 4 * 512 * 512  (4 because of k, q, v, o)     = 1M 
# #   FFNs:      2 * 512 * 1024 (c_fc, c_proj)                = 1M 

# # total: ~9.5M 

# # note that the above does not account for the LM_head, which typically gets swapped out depending on the downstream task. 
# # the LM_head, in this case, maps logits back to tokens
# # 512 * 10'000 = 5M    (A decent chunk compared to our 9.5M parameters)

# # NOTE: uncomment to see the model architecture 
# # model 

In [193]:
# for comparison. Note, wte should be (10000, 64)
# GPTNeoForCausalLM.from_pretrained('roneneldan/TinyStories-1M')

#### Training loop
Huggingface provides some powerful (and often confusingly long) APIs for model training. The `TrainingArguments` specifies our hyperparameters, which are used by the `Trainer` taking in the remaining objects (like `model`, `tokenizer`, and `train_dataset`). Specifically:

- `learning_rate` and `num_train_epochs` determine how much the model learns. A higher rate is faster, but more unstable. More epochs (entire passes over the dataset) yields incrementally better results, at the cost of more training time. 
- Batch sizes determine how many samples the model sees in *parallel*. Given `gradient_accumulation_steps=1` and a `batch_size=8`, the model will backpropagate the average loss of 8 samples; if `batch_size=1`, it will average the loss of `gradient_accumulation_steps` samples. It is important to make sure the backpropagated loss is averaged over the same number of samples, when comparing models. 

- `data_collator` batches (and optionally, pads) the input for the model. We have already padded in our `tokenized_dataset`, and leaving this argument empty will automatically batch the inputs. So why do we need it? 

    Glad you asked. This has to do with how the loss is computed in causal language modelling. In our case, we try to predict $p(y | x)$, where $x$ is an input sequence of tokens, and $y$ is the next token following that sequence. Our model, unaware of the target token $y$, outputs $\hat y$. 
    
    For `Trainer` to compute the (cross-entropy) loss, we need to provide it with both $y$ and $\hat y$. The `DataCollatorForLanguageModeling` knows this, and provides the next token $y$ as a separate part of the input, to the `Trainer`.

    The loss is the backbone of backpropagation, which we need to actually improve our model. If this is confusing, please re-watch Karpathy's GPT tutorial. 

If you prefer to write the training loop yourself, check out HF' `run_clm_no_trainer.py` scripts. 

In [194]:
train_dataset, eval_dataset = tokenized_dataset['train'], tokenized_dataset['validation']

batch_size = 16 # TinyStories claims 80, but I am training locally on my poor M1 Air
num_train_epochs = 1  # TinyStories doesn't mention
gradient_accumulation_steps = 16 # TinyStories claims 16

lr = 5e-4 # TinyStories claims 5e-4, higher values are preferable for smaller models

_train_steps = len(train_dataset) // (batch_size * gradient_accumulation_steps)
eval_steps = _train_steps // 10  # Evaluate every 10% of training steps


model_name = f'{model.num_parameters()//1e6:.1f}M-{config.num_layers}L-{config.num_heads}H-{config.hidden_size}C-{config.intermediate_size}I'

In [195]:
training_args = TrainingArguments(

    seed       = 42,
    use_cpu    = False, # use GPU if available (not necessarily faster on laptops, but Apple's MPS have good support)
    output_dir = f'./results/models/{model_name}',

    # NOTE: training params
    learning_rate    = lr,
    num_train_epochs = num_train_epochs,
    # Use a smaller batch size to fit into GPU RAM. 
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size  = batch_size,
    # You should aim to have the same amount of samples per acc step, in all of your experiments!
    # so, if you increase batch_size, decrease gradient_accumulation_steps by the same factor.
    gradient_accumulation_steps = gradient_accumulation_steps,

    # NOTE: Evaluation params
    # wandb is great for tracking experiments, it will even (try to) save your code nowadays
    evaluation_strategy = 'steps',
    eval_steps = eval_steps,
    save_steps = eval_steps,

    logging_first_step=True,
    logging_steps=max(1,eval_steps),
    report_to  = 'wandb',
)

trainer = Trainer(
    model = model, 
    args = training_args, 
    train_dataset = train_dataset, 
    eval_dataset = eval_dataset,
    data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

# print amount of training steps, and how often the model is evaluated
print(f'''
    training for {num_train_epochs} epochs, {len(train_dataset)} samples
    {batch_size} batch size, {gradient_accumulation_steps} accumulation steps
    gives {_train_steps} training steps.

    evaluating every {eval_steps} steps, {len(eval_dataset)} samples 
    ''')


    training for 1 epochs, 1000 samples
    16 batch size, 16 accumulation steps
    gives 3 training steps.

    evaluating every 0 steps, 100 samples 
    


Finally, we can train. 

This configuration takes ≤24hr to pre-train on my M1 Macbook Air with 16GB RAM. Python takes ≤4GB VRAM at a `batch_size=16` and ≤11GB at `batch_size=64`, though they take the same amount of time to train - likely because this processor is not designed to move that much data in and out of RAM constantly. And tbh, the GPU be lacking. If you decide to go the local-training-route, consider [chai](https://github.com/lvillani/chai) to keep your (Apple) laptop awake – there's probably a windows/linux equivalent too. 

In [196]:
# wandb.init(project='tinystories', name=model_name, config=training_args)
trainer.train()
trainer.save_model(f'./results/models/{model_name}')

  0%|          | 0/3 [05:25<?, ?it/s]

  0%|          | 0/3 [00:47<?, ?it/s]        

{'loss': 9.3204, 'grad_norm': 4.091865539550781, 'learning_rate': 0.0003333333333333333, 'epoch': 0.25}










                                       
                                             
  0%|          | 0/3 [00:49<?, ?it/s]


{'eval_loss': 8.472726821899414, 'eval_runtime': 1.6637, 'eval_samples_per_second': 60.106, 'eval_steps_per_second': 4.207, 'epoch': 0.25}



  0%|          | 0/3 [01:46<?, ?it/s]        

...

{'eval_loss': 8.11562728881836, 'eval_runtime': 13.56, 'eval_samples_per_second': 7.375, 'eval_steps_per_second': 0.516, 'epoch': 0.51}



  0%|          | 0/3 [02:58<?, ?it/s]        

{'loss': 8.1156, 'grad_norm': 2.2963554859161377, 'learning_rate': 0.0, 'epoch': 0.76}











                                       
                                             
  0%|          | 0/3 [03:02<?, ?it/s]

100%|██████████| 3/3 [02:31<00:00, 50.39s/it]


{'eval_loss': 7.976396560668945, 'eval_runtime': 4.1233, 'eval_samples_per_second': 24.252, 'eval_steps_per_second': 1.698, 'epoch': 0.76}
{'train_runtime': 151.1384, 'train_samples_per_second': 6.616, 'train_steps_per_second': 0.02, 'train_loss': 8.636452674865723, 'epoch': 0.76}


In [197]:
wandb.finish()

eval/loss,█▃▁
eval/runtime,▁█▂
eval/samples_per_second,█▁▃
eval/steps_per_second,█▁▃
train/epoch,▁▁▅▅███
train/global_step,▁▁▅▅███
train/grad_norm,█▃▁
train/learning_rate,█▅▁
train/loss,█▃▁
eval/loss,7.9764
eval/runtime,4.1233


## Inference!
Try out your own pre-trained model on some prompts

In [198]:
import textwrap # for pretty printing
w = textwrap.TextWrapper(replace_whitespace=False, break_long_words=False, width=60, initial_indent='   ', subsequent_indent='  ')
def see(text): print('\n\033[3m' + '\n\n'.join(['\n'.join(w.wrap(line))
                 for line in text.splitlines() if line.strip() != '']) + '\033[0m\n')

In [199]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = '9.0M-2L-4H-512C-1024I'
tokenizer = AutoTokenizer.from_pretrained('10k-gpt-neo')
tokenizer.pad_token_id = tokenizer.eos_token_id
model = AutoModelForCausalLM.from_pretrained(f'results/models/{model_name}')

In [201]:
prompt = 'Once upon a time, there was'
input_ids = tokenizer.encode(prompt, return_tensors='pt')

output = model.generate(input_ids, max_length=300, num_beams=1)
output_text = tokenizer.decode(output[0])

# textwrap with indentation on every new paragraph
see(output_text)


   Once upon a time, there was a
  a..,,,,,,,,,,,,,,,,,,,,,,,,,,........,,,,,,,,,,,....................................................................................................................................................................................................................................................

